In [ ]:
# Load modules necessary
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
import pandas as pd
from scipy.signal import detrend

%matplotlib qt

In [ ]:
# Creating a fake channel with a polynomial trend
fake_data = raw.get_data()[0]
t = np.linspace(0, 1, len(fake_data))

np.random.seed(42)
trend = np.random.randn(3)
fake_data += 4e-3 * np.polyval(trend, t)

# Let us look at different ways to remove trend from the data
# First, we can simply mean-center the data
# This is done by subtracting the mean of the data from the data
fake_data_centered = fake_data - np.mean(fake_data)

# Second, we can remove a linear trend from the data
# This can be done using detrend() function of numpy
fake_data_detrended = detrend(fake_data, type='linear')

# Next we can remove a polynomial trend from the data
# This is done by fitting a polynomial model to the data and subtracting the model from the data
# We can use the polyfit() function of numpy to fit a polynomial model to the data
# The polyfit() function returns the coefficients of the polynomial model
# We can use the polyval() function of numpy to evaluate the polynomial model at the data points
# We can then subtract the polynomial model from the data
trend = np.polyfit(t, fake_data, 3)
fake_data_detrended2 = fake_data - np.polyval(trend, t)

# Plot the original data and the detrended data
f, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].plot(t, fake_data)
axs[0, 0].set_title('Original Data')
axs[0, 1].plot(t, fake_data_centered)
axs[0, 1].set_title('Mean-Centered Data')
axs[1, 0].plot(t, fake_data_detrended)
axs[1, 0].set_title('Linear Detrended Data')
axs[1, 1].plot(t, fake_data_detrended2)
axs[1, 1].set_title('Polynomial Detrended Data')
plt.show()